In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
import sklearn.linear_model as skllm
%matplotlib inline

In [2]:
sales = pd.read_csv("kc_house_data.csv")
train_sales = pd.read_csv("kc_house_train_data.csv")
test_sales = pd.read_csv("kc_house_test_data.csv")
simple_features = ['sqft_living', 'bedrooms']
initial_weights = np.array([1.,4.,1.])

In [3]:
def get_features_matrix(dataframe, features, output):
  dataframe['constant'] = 1
  features = ['constant'] + features
  #print(features)
  features_matrix = dataframe[features].to_numpy()
  output_array = dataframe[output].to_numpy()
  #print(features_matrix)
  return (features_matrix,output_array)

(features_matrix, output) = get_features_matrix(sales, simple_features, 'price')
features_matrix

array([[   1, 1180,    3],
       [   1, 2570,    3],
       [   1,  770,    2],
       ...,
       [   1, 1020,    2],
       [   1, 1600,    3],
       [   1, 1020,    2]], dtype=int64)

In [4]:
def predict_outcome(features_matrix,weights):
    predictions = np.dot(features_matrix, weights)
    return (predictions)

normalize features matrix column wise so that the weights are not influenced by the value of the feature

calculating the 2-norm is simply calculating the sqrt 
of the magnitude of the vectorwhich can be done by calculating the dot-product 
and then taking a square root of the dot product 

In [6]:
def norms(features_matrix):
    norms_matrix = np.array([]) 
    normalized_features = np.array([])
    for i in range(len(features_matrix[0])):
        norms_matrix = np.append(norms_matrix, np.sqrt((np.dot(features_matrix[:,i],features_matrix[:,i]))))
        #print(norms_matrix.shape)
    normalized_features = features_matrix/norms_matrix
    #print(normalized_features.shape)
    #print(norms_matrix)
    #print(normalized_features.shape)
    return(normalized_features, norms_matrix)

In [6]:
(normalized_features, norm) = norms(features_matrix)
normalized_features
norm

array([1.47013605e+02, 3.34257264e+05, 5.14075870e+02])

estimate predictions by using the predict_outcome function defined above with normalized features, and initial weights

In [7]:
predictions = predict_outcome(normalized_features, initial_weights)

In [8]:
predictions

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])

for soft thresholding, where we set the weights
equal to ro[j] (which is the correlation of the 
prediction error from the model excluding jth feature, the jth feature, this correlation would be low if the model predicts well without j, but if doesn't then the model actually requires the jth feature which can be seen from the high correlation) + shifted by some value results in an prediction error using a model that does not include the jth feature. 

residuals of the model that does not include the jth feature 
and the jth feature this is ensured by adding weights[j]*feature_matrix[:,j] to the negative of predictions vector
if the model predicts very well without the jth feature
rho would be small, thus the weight on the jth feature would be smaller

here the feature_matrix is the normalized feature matrix


the lasso coordinate descent step function assigns weights on every
jth feature by first checking the ro[j] value

In [9]:
ro = [0 for i in range((normalized_features.shape)[1])]
for j in range((normalized_features.shape)[1]):   
    ro[j] = (normalized_features[:,j] * (output - predictions + (initial_weights[j] * normalized_features[:,j]))).sum()
print(ro)

[79400300.0145229, 87939470.82325175, 80966698.66623947]


In [10]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    predictions = predict_outcome(feature_matrix, weights)
    ro_i = (feature_matrix[:, i] * (output - predictions + (weights[i]*feature_matrix[:,i]))).sum()
    if i == 0: 
        new_weight_i = ro_i
    elif ro_i < - l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.: 
        new_weight_i =ro_i - l1_penalty/2.
    else: 
        new_weight_i = 0. 

    return (new_weight_i)

In [11]:
print(lasso_coordinate_descent_step(1, np.array([[3./sqrt(13),1./sqrt(10)],
                   [2./sqrt(13),3./sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


checking whether or not, ro[j] falls within the boundaries of a specific l1_penalty, if it does, then that value is knocked out or reduced to 0 (this just means that the correlation value on this jth feature is insignificant 

In [12]:
def in_l1range(value, penalty):
    return((value>=-penalty/2.) and (value<= penalty/2.))

In [13]:
for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print('for {} penalty: \n 1. Would the model knock out sqft_living? Ans: {} \n 2. Would the model knock out bedrooms? Ans: {}' .format(l1_penalty, in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty)))

for 140000000.0 penalty: 
 1. Would the model knock out sqft_living? Ans: False 
 2. Would the model knock out bedrooms? Ans: False
for 164000000.0 penalty: 
 1. Would the model knock out sqft_living? Ans: False 
 2. Would the model knock out bedrooms? Ans: True
for 173000000.0 penalty: 
 1. Would the model knock out sqft_living? Ans: False 
 2. Would the model knock out bedrooms? Ans: True
for 190000000.0 penalty: 
 1. Would the model knock out sqft_living? Ans: True 
 2. Would the model knock out bedrooms? Ans: True
for 230000000.0 penalty: 
 1. Would the model knock out sqft_living? Ans: True 
 2. Would the model knock out bedrooms? Ans: True


In [32]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights)
    delta = np.array(initial_weights) * 0.0 
    converged = False 
    while not converged: 
        for index in range(feature_matrix.shape[1]):
            #print('For Feature:', index)
            new_weight = lasso_coordinate_descent_step(index, feature_matrix, output, weights, l1_penalty)
            delta[index] = np.abs(new_weight - weights[index])
            #print('old weight:', weights[index])
            #print('new_weight:', new_weight)
            #print('change in weights', delta[index])
            #print('              ')
            weights[index] = new_weight
        max_change = max(delta)
        #print('max change is:', max_change)
        #print('               ')
        
        if max_change < tolerance:
            converged = True
    return weights 

In [33]:
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0
#features_matrix
#normalized_features
#norm

weights = lasso_cyclical_coordinate_descent(normalized_features, output, initial_weights, l1_penalty, tolerance)

In [34]:
weights

array([21624997.95951909, 63157247.20788956,        0.        ])

In [35]:
prediction = predict_outcome(normalized_features, weights)
residuals = output - prediction
RSS = np.dot(residuals, residuals)
print(RSS)

1630492476715386.5


In [36]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

target = 'price'

In [37]:
(multi_features_matrix, multi_output) = get_features_matrix(train_sales, all_features, target)
(normalized_multi_features, multi_norm) = norms(multi_features_matrix)

multi_initial_weights = np.zeros(len(all_features) + 1)
multi_l1_penalty1 = 1e7 
multi_tolerance = 1.0

In [38]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_multi_features, multi_output, multi_initial_weights, multi_l1_penalty1, multi_tolerance)
weights1e7

array([24429600.23440312,        0.        ,        0.        ,
       48389174.77154896,        0.        ,        0.        ,
        3317511.21492165,  7329961.81171425,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [39]:
multi_l1_penalty2 = 1e8
multi_tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalized_multi_features, multi_output, multi_initial_weights, multi_l1_penalty2, multi_tolerance)
weights1e8

array([71114625.71488702,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [42]:
multi_l1_penalty3 = 1e4
multi_tolerance2 = 5e5

weights1e4 = lasso_cyclical_coordinate_descent(normalized_multi_features, multi_output, multi_initial_weights, multi_l1_penalty3, multi_tolerance2)
weights1e4

array([ 78564738.34156762, -22097398.92430532,  12791071.87278517,
        93808088.09281193,  -2013172.75704954,  -4219184.93265014,
         6482842.81753506,   7127408.53480689,   5001664.8546964 ,
        14327518.43714051, -15770959.15237397,  -5159591.22213147,
       -84495341.7684364 ,   2824439.49703683])

To test these weights on the TEST data set we need to find the normalized features for the test data as well

In [44]:
(test_features_matrix, test_output) = get_features_matrix(test_sales, all_features, target)
(normalized_test_features, test_norm) = norms(test_features_matrix)

In [45]:
prediction_1e7 = predict_outcome(normalized_test_features, weights1e7)
residuals_1e7 = test_output - prediction_1e7
RSS1e7 = (residuals_1e7**2).sum()
RSS1e7

1568106721719153.2

In [46]:
prediction_1e8 = predict_outcome(normalized_test_features, weights1e8)
residuals_1e8 = test_output - prediction_1e8
RSS1e8 = (residuals_1e8**2).sum()
RSS1e8

1818706228108472.2

In [48]:
prediction_1e4 = predict_outcome(normalized_test_features, weights1e4)
residuals_1e4 = test_output - prediction_1e4
RSS1e4 = (residuals_1e4**2).sum()
RSS1e4

1855844223399372.0

instead of normalizing the test features, we can try normalizing the weights themselves

In [50]:
weights1e7_normalized = weights1e7 / multi_norm
weights1e8_normalized = weights1e8 / multi_norm
weights1e4_normalized = weights1e4 / multi_norm

In [54]:
prediction_1e7_2 = predict_outcome(test_features_matrix, weights1e7_normalized)
residuals_1e7_2 = test_output - prediction_1e7_2
RSS_1e7_2 = (residuals_1e7_2**2).sum()
RSS_1e7_2

275962075920366.78

In [55]:
prediction_1e8_2 = predict_outcome(test_features_matrix, weights1e8_normalized)
residuals_1e8_2 = test_output - prediction_1e8_2
RSS_1e8_2 = (residuals_1e8_2**2).sum()
RSS_1e8_2

537166151497322.75

In [56]:
prediction_1e4_2 = predict_outcome(test_features_matrix, weights1e4_normalized)
residuals_1e4_2 = test_output - prediction_1e4_2
RSS_1e4_2 = (residuals_1e4_2**2).sum()
RSS_1e4_2

228459958971393.25